### IMPORTANT: For all visuals, please hover over the chart to see label values. 

-----------------------------------

## Importing libraries and config

In [1018]:
import warnings
warnings.filterwarnings("ignore")

In [1007]:
import pandas as pd
import numpy as np
import re
from matplotlib.pyplot import figure
from IPython.display import display,HTML
import cufflinks as cf
import plotly.express as px
from nltk import word_tokenize

In [1009]:
cf.set_config_file(theme='ggplot',sharing='public',offline=True)

## Useful function definitions

In [ ]:
def summarize_data(data):
    
    # Create a descriptive statistics dictionary
    dic = {}
    dic['dtypes'] = data.dtypes
    dic['count'] = data.count()
    dic['null_sum'] = data.isnull().sum()
    dic['null_pct'] = data.isnull().mean()
    dic['nunique'] = data.nunique()
    dic['min'] = data.min()
    dic['25%'] = data.quantile(0.25)
    dic['50%'] = data.quantile(0.5)
    dic['75%'] = data.quantile(0.75)
    dic['max'] = data.max()
    dic['mean'] = data._get_numeric_data().mean()
    dic['median'] = data._get_numeric_data().median()
    dic['std'] = data._get_numeric_data().std()
    dic['skew'] = data._get_numeric_data().skew()
    
    # Convert dictionary to readable DataFrame
    eda = pd.DataFrame(dic)
    eda = eda.fillna('-').round(3)
    
    return eda

def explore(data, method="summarize", is_large_dataset=False):

    
    try:
        
        # Default method
        if method=="default":
            boxplot_of_numerical_features(data)
            histogram_of_numerical_features(data, bins)
            
        if method=="correlation":
            return get_correlation_between_numerical_features(data)
    
    except Exception as e:
        print(e)

def get_correlation_between_numerical_features(data):
    
    print("Correlation Analysis - Heatmap")
    
    # Retreive correlation matrix
    corr = data.corr()
    
    return corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)       

def eda_num(data, method="default", bins=10):
    
    try:
            
        if method=="correlation":
            return get_correlation_between_numerical_features(data)
    
    except Exception as e:
        print(e)
        
def eda_timeseries(data, x, y):
    
    try:
        print("Distribution - Time Series Data")
        plot_title = "Distribution of "+y+" across "+x
        data = go.Scatter(x=data[x], y=data[y])
        layout = go.Layout(title=plot_title, xaxis=dict(title=x), yaxis=dict(title=y))
        fig = go.Figure(data=[data], layout=layout)
        plotly.offline.iplot(fig)
        
    except Exception as e:
        print(e)

def clean_text(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

## Reading data and defining functions

In [4]:
data = pd.read_json("data_clean.json")

## The imbalance in claim_types raises concerns. Upon further study, a data shift is realised and hence further in the cleaning process we need to filter by "ADLD" and "EW". However, again upon further inspection, we can see that "EW" are corresponding to the test cases defined by OneAssist and hence should be ignored.

In [956]:
pd.DataFrame(data)["claim_type"].value_counts().iloc[:5]

ADLD    2023
10       153
2         48
1         39
27        33
Name: claim_type, dtype: int64

## But since I have defined a snippet that deals with test-cases, "EW" has been left here for generalisation purposes as it is possible that future data might not have tests corresponding to the claim-type "EW".

In [7]:
new = data[(data.claim_type == "ADLD") | (data.claim_type == "EW")]

In [12]:
new.shape

(2025, 96)

## Null percentages for each column is calculated and columns with more than 80% Null values are filtered out. 
### [Note: The change was verified by having a sampled look at the data to ensure important features are not deleted.]

In [13]:
null_percentages = ((new.isna().sum()/new.shape[0]) * 100).reset_index()

In [14]:
new_1 = new[new.columns[new.isna().mean() < 0.8]]

In [16]:
new_1.shape

(2025, 64)

## Removal of Test cases from the dataset

In [31]:
new_1["incident_desc"].isna().sum()

171

In [1109]:
new_1["incident_desc"] = new_1["incident_desc"].fillna(value=pd.np.nan)
new_1["incident_desc"] =new_1["incident_desc"].astype(str)

In [37]:
test_removal = ["TEST,test,testing,TESTING"]

new_1 = new_1[~new_1['incident_desc'].str.contains('test')]

In [624]:
new_1 = new_1[~new_1.close_claim_reason_code.astype(str).str.contains("test")]

In [39]:
new_1.shape

(2018, 64)

## After filtering out values, it is imperative to assign datetime columns their natural datatype. 
### [Sidenote: Admissibility date had garbled values in place of the year, however this column had already been phased out in the NULL filter]

In [17]:
[col for col in new_1.columns if "time" in col]

['damage_loss_datetime',
 'damage_loss_datetime_ver_stat',
 'last_updated_timestamp']

In [18]:
[col for col in new_1.columns if "date" in col]

['actual_delv_date',
 'actual_repair_date',
 'claim_int_date',
 'created_date',
 'damage_loss_datetime',
 'damage_loss_datetime_ver_stat',
 'estimation_completed_date',
 'last_updated_timestamp',
 'modified_date',
 'pickup_date',
 'proposed_repair_date']

In [533]:
dtimes = new_1.filter(regex="date|time").drop(columns="damage_loss_datetime_ver_stat").apply(pd.to_datetime, errors="coerce")

## Since not all columns would be important or implicitly of a datetime nature, they have been dropped.

In [ ]:
new_1.filter(regex="date|time").drop(columns="damage_loss_datetime_ver_stat").update(dtimes.columns)

## Removing columns with high cardinality and only constants (but with intuition).

In [41]:
categorical_columns = [col for col in new_1.columns if new_1[col].dtype == object]
columns_with_high_cardinality = [col for col in categorical_columns if new_1[col].nunique() > 1000]

In [42]:
columns_with_high_cardinality

['activiti_proc_id',
 'claim_asset_attributes',
 'claim_int_date',
 'created_by',
 'created_date',
 'damage_loss_datetime',
 'incident_desc',
 'last_updated_timestamp',
 'modified_date',
 'serial_number']

In [44]:
new_1 = new_1.drop(columns=["activiti_proc_id","serial_number","svc_id"], axis=1)

### A quick glance at the dataset tells us that the columns below have a constant value, hence we may cannot derive substantial value out of them. 
#### all_doc_sent_ic has constant value "True"	
#### courtesy_raised has constant value ""	
#### ic_reinstatement_prem has constant value "0.0"	
#### ic_under_insur has constant value "0.0"	
#### in_query_flag has constant value "False"
#### repair_cost_to_customer 

In [45]:
new_1 = new_1.drop(columns=["all_doc_sent_ic","courtesy_raised","ic_reinstatement_prem","ic_under_insur","in_query_flag","repair_cost_to_customer","claim_type"], axis=1)


## All ID related columns have been dropped to ensure we get the best summarised view

In [175]:
new_1 = new_1.drop(columns =["level_0", "df_index","account_no", "admissibility_assignee",'claim_pk','cust_id','default_service_cntr_id','delivery_partner_id','estimate_cost_to_customer','fulfilment_hub_id','hub_id','service_cntr_id','sr_no','ic_approval_assignee'])
                     

## IC (assuming "insurance-company") related columns sliced into another dataframe for a holistic perspective

In [720]:
ic_df = new_1[[ 'ic_amount_to_cust','ic_depriciation','ic_estimate_amount','ic_excess_amt','ic_market_value','ic_payment_amt','ic_salvage_amt']]

In [724]:
ic_df = ic_df.reset_index(drop=True)

# Claim Asset Attributes

In [49]:
new_1["claim_asset_attributes"][0]

'damageType|Physical Damage~damagePlace|PUNJ PEER ROAD~purchaseInvoice|~otherDamagePart|~damageIsTouchWorks|N~damageIsSwitchedOn|Y~damagePart|Front touch screen~paymentReqId|9632211'

## Claim asset data needs to be split into a different set of frame since it has multiple fields of information withing itself. This data will then be merged with the master dataset (new_1) to realise the damage attributes for a given case number.

In [50]:
new_1 = new_1.reset_index()

list_caa = []
for i in range(len(new_1["claim_asset_attributes"])):
    val_caa = pd.Series(new_1["claim_asset_attributes"][i].split("~")).str.split("|")
    to_add_col = pd.DataFrame(val_caa).explode(column=0).iloc[::2][0]
    list_caa.append(pd.DataFrame(val_caa).explode(column=0).iloc[1::2].T)

dfcaa = pd.concat(list_caa)

In [51]:
dfcaa.columns = to_add_col

dfcaa = dfcaa.replace(r'^\s*$', np.nan, regex=True)

## Creating an independant slice of claim-attributes for better clarity

In [1080]:
claim_attributes = pd.concat([dfcaa.reset_index(drop=True),new_1[["account_no","assignee","verification_sub_status"]] ], axis=1)

In [59]:
claim_attributes

,damageType,damagePlace,purchaseInvoice,otherDamagePart,damageIsTouchWorks,damageIsSwitchedOn,damagePart,paymentReqId,account_no,assignee,verification_sub_status
0,Physical Damage,PUNJ PEER ROAD,NaN,NaN,N,Y,Front touch screen,9632211,3965800.0,Arv-BharatBharwad,Approved
1,Physical,anwalkhera,NaN,NaN,Y,Y,Front touch screen,NaN,3965810.0,Arv-KhalidKhan,None
2,Physical Damage,road,NaN,NaN,N,N,Others,NaN,3971473.0,Arv-MariyaKhan,Approved
3,Physical,at home,NaN,NaN,Y,Y,Front touch screen,NaN,3971528.0,Arv-KamleshGupta,IC Rejected
4,Physical Damage,on Road,NaN,NaN,N,N,Front touch screen,NaN,3971744.0,DEFAULT_CMS_USER,Approved
...,...,...,...,...,...,...,...,...,...,...,...
2013,Liquid Damage,road,NaN,NaN,N,N,NaN,9827089,5266696.0,skazi,Approved
2014,Physical,road,NaN,NaN,N,Y,Front touch screen,NaN,5274799.0,Arv-BikramjeetNatt,None
2015,Physical,GALOBAL CITY KHARAR,NaN,NaN,N,N,Front touch screen,NaN,5283200.0,Arv-ShenugaArundudhi,None
2016,Physical Damage,Me apni duty khatam Kar ke bike se Ghar ja rh...,NaN,NaN,N,Y,Front touch screen,9878986,5357342.0,Arv-SandhyaVishwakar,Approved


In [100]:
# new_1.to_csv("data/master.csv")
# freq.stack().value_counts().reset_index().to_csv("data/frequencies.csv")
# claim_attributes.to_csv("data/claim_att.csv")

# Incident Descriptions

## Calculating word frequencies for each description.
### - Firstly, I created a list of words to mitigate the jumbled language ["jbb,kl,m,k,etc"]. This is the filter file, named "stop_hinglish.txt".
### - This filter file has been ingested and then the words (after being tokenized) are filtered and then converted to lowercase. 
### - After the steps, frequency of occurence of these words are mapped out into a nested list which is then parsed into a dataframe for ease of use.
### This data can be mapped with the respective claim ID or codes and therefore be used to analyze summarised reasons based on keyword density. 

## Words like phone, mobile, screen, display, damage etc have been added to the filter list since the nature of gadget and it's parts are obvious. And proper details of the damage are listed under Claim attribute assets, since we are looking for most probable causes, those words are not needed.

In [999]:
stop = list(pd.read_csv('stop_hinglish.txt', delimiter = "\t",header=None)[0])

In [1000]:
new_1 = new_1.reset_index()

In [1001]:
word_list = []
for i in range(len(new_1.incident_desc)):
    tokens = word_tokenize(clean_text(new_1.incident_desc[i]))
    words = [word for word in tokens if word.isalpha()]
    filtered_sentence = [w for w in words if w.lower() not in stop]
    word_list.append(str(filtered_sentence).lower())


## The entire description set is cleaned off braces, punctuations and assigned to a dataframe.

In [1002]:
freq = pd.DataFrame(word_list)[0].map(lambda x: x.strip('[,{)').replace("'","").split(',')).apply(pd.Series).T

## The frequency dataframe is melted and the top words are extracted. 
## The entire list can be found inside the variable frequency counts where value is the count and index is the word.

In [1003]:
frequency_counts = freq.melt().value.value_counts().reset_index()

In [1004]:
freq.melt()["value"].value_counts().iloc[:15]

 bike       1819
 gir        1524
 road        852
 pocket      792
 achanak     497
 takkar      417
 pant        335
 time        309
 nikal       298
 hath        294
 pani        283
 car         243
 jeb         239
 slip        222
 call        222
Name: value, dtype: int64

## The most probable causes evident in decreasing manner of counts from the frequency of words are 
## - Bike related causes (due to rider or third party, both.)
## - Falls [on the road (from a bike) or from the pocket or while on call]
## - Collision
## - Slips when removing the device from pocket
## - Water-contact
## - Drops inside vehicle (car/bus)


In [1005]:
freq.melt()["value"].value_counts().iloc[1:15].iplot("bar")

In [96]:
claim_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018 entries, 0 to 2017
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   damageType               1847 non-null   object 
 1   damagePlace              2018 non-null   object 
 2   purchaseInvoice          0 non-null      float64
 3   otherDamagePart          0 non-null      float64
 4   damageIsTouchWorks       1832 non-null   object 
 5   damageIsSwitchedOn       1832 non-null   object 
 6   damagePart               1801 non-null   object 
 7   paymentReqId             933 non-null    object 
 8   account_no               2018 non-null   float64
 9   assignee                 2018 non-null   object 
 10  verification_sub_status  1415 non-null   object 
dtypes: float64(3), object(8)
memory usage: 173.5+ KB


## List of numeric columns from the dataframe.

In [179]:
list(new_1._get_numeric_data())

['amt_paid_by_oa_to_asc',
 'estimate_amount',
 'estimate_cost_to_company',
 'ic_amount_to_cust',
 'ic_depriciation',
 'ic_estimate_amount',
 'ic_excess_amt',
 'ic_market_value',
 'ic_payment_amt',
 'ic_salvage_amt',
 'pickup_partner_id',
 'repair_cost_to_company']

In [184]:
new_1.describe()

,amt_paid_by_oa_to_asc,estimate_amount,estimate_cost_to_company,ic_amount_to_cust,ic_depriciation,ic_estimate_amount,ic_excess_amt,ic_market_value,ic_payment_amt,ic_salvage_amt,repair_cost_to_company
count,1320.000000,1319.000000,1319.000000,1320.000000,944.000000,1134.000000,1298.000000,724.000000,522.000000,1298.000000,1088.000000
mean,5777.038636,6221.612585,5532.909022,1466.048485,680.582627,4449.409171,603.309707,3765.023481,1665.975096,0.423729,1791.999081
std,4453.995332,4730.800484,4698.199364,3453.107388,1316.967367,3142.470419,300.142081,6000.611529,3216.756010,11.185233,2214.512087
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3595.000000,3595.000000,3047.000000,0.000000,0.000000,3361.000000,500.000000,0.000000,0.000000,0.000000,150.000000
50%,4747.500000,4889.000000,4177.000000,0.000000,0.000000,4047.000000,600.000000,0.000000,0.000000,0.000000,650.000000
75%,7522.750000,7962.500000,6932.500000,0.000000,0.000000,5781.000000,750.000000,8990.000000,2610.250000,0.000000,3300.000000
max,44200.000000,46677.000000,56860.000000,19494.000000,8997.000000,27400.000000,3095.000000,29990.000000,18743.000000,350.000000,32184.000000


In [212]:
numeric_new_1 = clean(new_1._get_numeric_data(), method='replaceval',to_replace=np.nan, value=0)

## Glancing through the below stats, ic_salvage_amt is the one with highest skew where the max value is really really high but the 75% of the data is 0.
## The amount metrics[amt paid to oa to asc, estimate amt, estimate ctc] which from a top view hold a very important view are explored further in the notebook but are in close vicinity indicating that they are correlated and or associated. This is further confirmed when we plot the heatmap.
## The market_value and depreciation are also correlated, this dependancy could have been used to trim features for a model based on pragmatic usefulness, however, for EDA these are important for a summarised view.

## Many features depreciation, amount to cust, salvage can be considered not apt for numerical analysis or inference given their sparsely populated nature. Same can be verified by the universal boxplots and histograms below.

In [1082]:
numeric_new_1.corr().iplot("heatmap")

In [215]:
## summary
explore(numeric_new_1,method="summarize")

,dtypes,count,null_sum,null_pct,nunique,min,25%,50%,75%,max,mean,median,std,skew
amt_paid_by_oa_to_asc,float64,2018,0,0.0,692,0.0,0.0,3520.0,5662.75,44200.0,3778.836,3520.0,4530.735,2.091
estimate_amount,float64,2018,0,0.0,672,0.0,0.0,3595.0,5946.00,46677.0,4066.555,3595.0,4836.572,1.810
estimate_cost_to_company,float64,2018,0,0.0,690,0.0,0.0,3046.0,5169.50,56860.0,3616.406,3046.0,4621.455,2.824
ic_amount_to_cust,float64,2018,0,0.0,171,0.0,0.0,0.0,0.00,19494.0,958.961,0.0,2878.209,3.123
ic_depriciation,float64,2018,0,0.0,124,0.0,0.0,0.0,0.00,8997.0,318.370,0.0,962.421,3.219
ic_estimate_amount,float64,2018,0,0.0,472,0.0,0.0,0.0,4425.00,27400.0,2500.312,0.0,3228.447,1.633
ic_excess_amt,float64,2018,0,0.0,99,0.0,0.0,475.0,650.00,3095.0,388.056,475.0,376.165,0.620
ic_market_value,float64,2018,0,0.0,116,0.0,0.0,0.0,0.00,29990.0,1350.781,0.0,4021.153,3.043
ic_payment_amt,float64,2018,0,0.0,128,0.0,0.0,0.0,0.00,18743.0,430.941,0.0,1790.335,4.883
ic_salvage_amt,float64,2018,0,0.0,3,0.0,0.0,0.0,0.00,350.0,0.273,0.0,8.972,34.872


# Please double-click the variablename to display the respective boxplot. Click another variable to compare. Reclick the selected variable to deselect. (plotted by plot.ly)
## This applies to all universal plots in the notebook, if visible.

In [266]:
numeric_new_1.iplot(kind='box',title="Value Distribution")

In [265]:
numeric_new_1.iplot(kind='hist',histfunc='avg',bins=15,title="Average Value by Metric")

## Below is substitution of Physical and Liquid, to values with "damage" word added. This is done to preserve the uniform nature of values

In [273]:
claim_attributes.damageType = claim_attributes.damageType.replace(to_replace=["Physical","Liquid"], value=["Physical Damage","Liquid Damage"])


## We can see that the Physical (screen,entire display,frame,etc) damages far outweigh the Liquid damages. As liquid damages only make up only about ~25% of what Physical damage counts amount to.

In [274]:
claim_attributes.damageType.value_counts().sort_values(ascending=True).iplot(kind="barh",title="Types of Damages")

## Most affected part is the touchscreen (1415), which is no surprise considering it is also the most fragile part. Buttons (8) being usually of plastic and metal nature with rubber inserts rarely make up for damages but they are still there nonetheless.
## Backglass, camera and buttons together make up for about 86 damage counts from the list.

In [275]:
claim_attributes.damagePart.value_counts().loc[lambda x: x>1].sort_values(ascending=True).iplot(kind="barh", title="Most Affected Part")


## A closer holistic look tells us that most damages take place on 
# the road, followed by home, market/shops and also bus stops and offices in the same order as mentioned. [decreasing order of popularity for mobile phone damages]

### Rest of the values are specific addresses but they can overall still be considered under the same causes as above.

In [1084]:
placelist = list(claim_attributes.damagePlace.str.lower().value_counts().reset_index()["index"])

## Exhaustive list stored in placelist variable

In [1087]:
placelist[:30]

['road',
 'home',
 'road ',
 'at home',
 'on road',
 'on road ',
 'at road',
 'home ',
 'at bus stop',
 'market',
 'at home ',
 'at park',
 'on a road ',
 'agra',
 'office',
 'kanpur',
 'bus stop',
 'on the road ',
 'on the road',
 'my home',
 'at road site ',
 'at market',
 'market ',
 'varanasi',
 'jalandhar road hoshiarpur',
 'road,lucknow',
 'road, agra',
 'kidwai nagar',
 'at road ',
 'transport nagar']

## The Approval and Rejection rates linearly scale towards the number of cases with respective damages.

In [1088]:
pd.crosstab(claim_attributes['damagePart'],claim_attributes['verification_sub_status']).iloc[1:].iplot("bar",orientation="h",title="Status based on type of damage parts")


## About ~8% of Physical damages are rejected while about ~9% of Liquid damages are rejected.

In [1092]:
pd.crosstab(claim_attributes['damageType'],claim_attributes['verification_sub_status']).iloc[:].iplot("bar",barmode="stack",orientation="h",title="Status based on damage type")


## Below is substitution of "Y" and "N" with "Yes" and "No". This is done to preserve the uniform nature of values

In [439]:
claim_attributes.damageIsSwitchedOn = claim_attributes.damageIsSwitchedOn.replace(to_replace=["Y","N"], value=["Yes","No"])


## ~7% of devices not in working condition receive a rejection while ~9% of those working receive rejection. This from an empirical point of view can be attributed mostly negligence.
## Approval rates are well above 88% for both cases wth pending and re-appeals forming really small no of cases.

In [441]:
pd.crosstab(claim_attributes['damageIsSwitchedOn'],claim_attributes['verification_sub_status']).iloc[:].iplot("bar",title="Status based on device working condition")


## Tabular values for assignees and their case status distributions. 
[DEFAULT_CMS_USER correspondence to assignee unclear]

In [452]:
pd.crosstab(claim_attributes['assignee'],claim_attributes['verification_sub_status']).sort_values(by="Approved",ascending=False).iloc[:]


verification_sub_status,Approved,IC Rejected,Pending with IC,Re-appeal
assignee,,,,
DEFAULT_CMS_USER,435,0,5,0
Arv-MinakshiMudliyar,171,2,2,0
Arv-BharatBharwad,160,1,3,0
Arv-MariyaKhan,95,5,3,0
Arv-MadhuriGaikwad,94,3,0,0
Arv-EktaPawar,72,0,1,0
Arv-OmkarPange,65,0,1,0
Arv-SandhyaVishwakar,39,0,0,0
Arv-KamleshGupta,20,1,0,0


## Time metrics

## Estimated time taken has been defined in the time_taken_est while the time_taken_act is the actual time taken.

In [947]:
time_taken_est = pd.DataFrame(dtimes.estimation_completed_date - dtimes.created_date)[pd.DataFrame(dtimes.estimation_completed_date - dtimes.created_date)[0]>pd.Timedelta(0)]
time_taken_act = pd.DataFrame(dtimes.actual_delv_date - dtimes.created_date)[pd.DataFrame(dtimes.estimation_completed_date - dtimes.created_date)[0]>pd.Timedelta(0)]


In [950]:
time_taken_act[0].dt.days.iplot(title="Average completion day (trend)",yTitle="Days")


## [Below data is the calculated time from created to delivery. ]
## Estimates of time: It can be seen that with a standard deviation of 49 days it takes an estimated average of ~85 days for a resolution. Median is 78.
## Actual time: For actual number, the average stands at ~81 days for settlement, but the data median is 82 days.
## Min number of days is 3 for both. However, the 75th percentile and max values are slightly higher for estimates and hence the higher mean.

## [Mean calculation below is based on the entire claim settlement given "x" number of claims and "y" number of days in general.]

In [1101]:
time_taken_est.shape[0],time_taken_act.shape[0]

(1196, 1196)

In [954]:
print((time_taken_est[0].dt.days.sum()/time_taken_est.shape[0]))
print((time_taken_act[0].dt.days.sum()/time_taken_act.shape[0]))

85.49247491638796
80.76421404682274


In [1102]:
time_taken_act[0].dt.days.iplot("box",yTitle="Time taken (days) for a case", xTitle="Spread", orientation="h")


In [952]:
time_taken_est[0].dt.days.iplot("hist",xTitle="Days",title="Estimated time for settlement")
time_taken_act[0].dt.days.iplot("hist",xTitle="Days",title="Actual Time taken for settlement")

## Calculation of settlement in days

In [946]:
dtimes.actual_delv_date - dtimes.created_date

0      221 days 11:35:49.150000
1                           NaT
2      196 days 10:17:31.663000
3                           NaT
4       42 days 10:12:36.282000
                 ...           
2013    35 days 09:27:10.497000
2014                        NaT
2015                        NaT
2016                        NaT
2017                        NaT
Length: 2013, dtype: timedelta64[ns]

In [623]:
new_1.shape[0]

2013

## "New_with_diff" contains columns from new_1 and the above calculated settlement days.

In [737]:
new_with_diff = pd.concat([new_1.reset_index(drop=True),(dtimes.estimation_completed_date - dtimes.created_date).dt.days.reset_index(drop=True)],1)


In [738]:
new_with_diff = new_with_diff.rename(columns={0:"Time_taken"})

## "New_with_nodates" removes all the datetime columns for problem free grouping and is further sliced into "Slice_cost"

In [740]:
new_with_nodates = new_with_diff.drop(columns=dtimes.columns)

In [742]:
slice_cost = new_with_nodates[["claim_status","close_claim_reason_code","device_warranty","estimate_cost_to_company","repair_cost_to_company","Time_taken","verification_sub_status"]]

## It can be seen that average time taken for a case to be defined as a rejected case (~60 days) is the lowest while it is highest for the resolved cases. Which can be pragmatically inferred given the fact that resolution takes time, unresolved cases somehow also manage to add-on considerable amounts of days to the claim processes (on avg about ~85 days).

In [631]:
slice_cost.groupby("claim_status")["Time_taken"].mean().iplot(title="Claim Status vs Time Taken for them respectively ")


## Most of the rejection reasons revolve around non-receipt of "documents" and "claim_not_required" which form about ~365 cases combined.

## Negligence along with improper handling also plays a part for rejection reasons with 47 cases and other reasons are claim_already_raised, not_covered, same_damages form the rest of the bunch.

In [1107]:
slice_cost.close_claim_reason_code.value_counts().reset_index().rename(columns={"index":"reason","close_claim_reason_code":"occurences"}).iloc[:20]


,reason,occurences
0,CCR3_DOCUMENTS_NOT_RECEIVED,206
1,CCR1_CLAIM_NOT_REQUIRED,110
2,CCR19_DELAY_IN_CLAIM,48
3,Documents not received,40
4,CCR32_RECLAIMING_FOR_THE_SAME_DAMAGES,40
5,CCR20_NEGLIGENCE,34
6,CCR34_CAUSE_OF_LOSS_NOT_COVERED_UNDER_THE_POLICY,20
7,CCR8_R\/NC_3_ATTEMPTS,13
8,CCR21_IMPROPER_HANDLING,13
9,1CLAIM_ALREADY_RAISED,10


## It can be seen that ~8% of devices in warranty are rejected, while devices not in warranty form a very very small number.

In [1108]:
pd.crosstab(slice_cost['device_warranty'],slice_cost['verification_sub_status']).iloc[:].iplot("barh",barmode="stack",title="Approvals vs Warranty")


## Average repair ctc lies around 1.8k for devices in warranty while it is about 829 for devices not covered in warranty.

In [912]:
slice_cost.groupby("device_warranty")["repair_cost_to_company"].mean().iloc[1:].iplot("bar",title="Average repair costs to company",xTitle="Warranty Status",yTitle="Avg. Cost in Rs.")

In [683]:
slice_cost[["estimate_cost_to_company","repair_cost_to_company"]].mean().iplot("bar")

In [1079]:
ic_df.head()

,ic_amount_to_cust,ic_depriciation,ic_estimate_amount,ic_excess_amt,ic_market_value,ic_payment_amt,ic_salvage_amt
0,0.0,NaN,4047.0,650.0,NaN,NaN,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3900.0,1800.0,NaN,300.0,6000.0,NaN,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7799.0,2200.0,0.0,1000.0,10999.0,7749.0,0.0


## Max repair ctc is 32.1k which is just about 43.3% to the estimated max ctc of ~57k. And the general trend says that the repair costs are on average ~32% lesser than that of the estimated costs.
## It can also be noted that actual ctc is highly concentrated towards the lower numbers and only about 25% costs are above 3.3k while the estimates say that 75% of the costs are above 3k, nice.
## The means are also highly different. 
## In general the spread of estimate ctc is high while the actual repair costs are skewed.

## For insurance_company (ic), estimates are actually very close to the actual payment. But the actual payment amount has far lesser values than the estimates so we are relying on a sample.
## It is also interesting to note that, 50% of the values in actual ic_fields are 0 and only the other 50% has higher values. But since these are actual payments we can scale up and have a proper look.

In [1078]:
ic_df[["ic_estimate_amount","ic_payment_amt"]].describe()

,ic_estimate_amount,ic_payment_amt
count,1134.000000,522.000000
mean,4449.409171,1665.975096
std,3142.470419,3216.756010
min,0.000000,0.000000
25%,3361.000000,0.000000
50%,4047.000000,0.000000
75%,5781.000000,2610.250000
max,27400.000000,18743.000000


In [1073]:
slice_cost[["estimate_cost_to_company","repair_cost_to_company"]].describe()

,estimate_cost_to_company,repair_cost_to_company
count,1319.000000,1088.000000
mean,5532.909022,1791.999081
std,4698.199364,2214.512087
min,0.000000,0.000000
25%,3047.000000,150.000000
50%,4177.000000,650.000000
75%,6932.500000,3300.000000
max,56860.000000,32184.000000


In [749]:
pd.concat([slice_cost[["estimate_cost_to_company","repair_cost_to_company"]],ic_df[["ic_estimate_amount","ic_payment_amt"]]],1).iplot("box",subplots=True)


In [750]:
slice_cost["estimate_cost_to_company"].iplot("hist",bins=15,title="Estimate Costs")
slice_cost["repair_cost_to_company"].iplot("hist",bins=15,title="Actual Costs")
ic_df.ic_estimate_amount.iplot("hist",bins=15,title="IC Estimate Costs")
ic_df.ic_payment_amt.iplot("hist",bins=15,title="IC Estimate Costs")


## "ts_month_status" contains a crosstab of creation dates and verification statuses. The month order is set in "new_order" list to later order the months by chronology.

In [853]:
new_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']


ts_month_status = pd.crosstab(pd.to_datetime(new_with_diff.created_date).dt.month_name(),new_with_diff.verification_sub_status)


## Case status numbers by respective months denote highest number of cases rejected by insurance company are in March and May (23,24 respectively) while highest "Approved" cases come from March and April. 
## The ratio of highest approvals also can be found in March and April relative to the number of cases.

In [1070]:
ts_month_status.reindex(new_order, axis=0).iplot(title="Case status numbers by Month", xTitle="Months",yTitle="Case Counts")


### ts_month_costs pairs repair_ctc with created_date for the claim.

In [818]:
ts_month_costs = pd.concat([pd.to_datetime(new_with_diff.created_date).dt.month_name(),new_with_diff.repair_cost_to_company],1)
ts_month_costs.head(2)

,created_date,repair_cost_to_company
0,January,3180.0
1,January,NaN


## Considering the total sum, March seems to be the most cost-heavy month for the company. This is also reiterated by the number of initiations in the month. When we look at the no of claims and total expenditure, highest CTCs are obvious to trace.

### "new_for_acc" is a slice of the master dataset "new" for getting the original (w/test case) claim counts

In [1067]:
new_for_acc = new[~new.close_claim_reason_code.astype(str).str.contains("test")]
accounts = new_for_acc[["account_no","created_date"]]
accounts["created_date"] = pd.to_datetime(accounts["created_date"]).dt.month_name()

In [1068]:
accounts.groupby("created_date")["account_no"].count().sort_values(ascending=False)

created_date
March       431
April       338
February    272
June        270
January     260
May         258
July        158
August       32
Name: account_no, dtype: int64

In [861]:
ts_month_costs.groupby("created_date")["repair_cost_to_company"].sum().sort_values(ascending=False).iplot(title="Total Repair costs to company by Month",xTitle="Months",yTitle="Total Costs")


## While it is useful to see the total cost to company by respective months, it is important to realise that the months have variable settlement counts and hence, median cost would be much more useful in this case.
## In which case, June and July are the most expensive mean and median spend months while March surprisingly is one of the three lowest spend months. 
## This denotes that overall a lot of lower amounts were given out in March and they added up to a much greater cost than higher average and median spend months.


In [1044]:
ts_month_costs.groupby("created_date")["repair_cost_to_company"].mean().sort_values(ascending=False).iplot(title="Average Repair costs to company by Month",xTitle="Months",yTitle="Total Costs")

ts_month_costs.groupby("created_date")["repair_cost_to_company"].median().sort_values(ascending=False).iplot(title="Median Repair costs to company by Month",xTitle="Months",yTitle="Total Costs")


## The average amount paid by OA to ASC range from 1.3k to 44.2k and the general spread is right skewed given the mean and median values. There are few outliers in the field but these cannot be ignored or substituted.

In [1033]:
new_with_diff["amt_paid_by_oa_to_asc"].describe()

count     1320.000000
mean      5777.038636
std       4453.995332
min          0.000000
25%       3595.000000
50%       4747.500000
75%       7522.750000
max      44200.000000
Name: amt_paid_by_oa_to_asc, dtype: float64

In [1040]:
new_with_diff["amt_paid_by_oa_to_asc"].iplot("hist",title="Payouts from OA to ASC",xTitle="Amount")

## Repair Estimates are working really well since we can see that overall there have been no cases where the proposed repair date has been before the actual repair date. This is a good if the estimates where generated by a model, which, in this case is working really well.
## If the proposed dates have been assigned by the assignee's then it is commendable to note that there 100% of the resolutions have been faciliated on time.

In [777]:
(pd.to_datetime(new_with_diff["proposed_repair_date"]) - pd.to_datetime(new_with_diff["actual_repair_date"])).dt.days.iplot("hist",title="Repair Estimates",xTitle="Diff between proposed and actual repair dates (in days)",yTitle="Day Counts")


## Claims in Feb in general have the largest variance followed by March (highest claims month). Aug has the lowest initiation. In general a trend of decreasing claim TAT can be followed which may be attributed to the skew of data.

In [890]:
eda_timeseries(new_with_diff,x="created_date",y="Time_taken")

Distribution - Time Series Data


## Most claim initiations happened in March at 431 about 22% higher than April (next month) and about 36.8% more than Feb (previous month).
## Lowest claim initiations can be found in July (32) and August (158) which can possibly be attributed to this sample of data. As can the fact that some months do not have initiations.

In [1061]:
pd.to_datetime(new.created_date).dt.month_name().value_counts()

March       431
April       338
June        273
February    273
January     260
May         259
July        158
August       32
Name: created_date, dtype: int64

In [1062]:
accounts.groupby("created_date")["account_no"].count().sort_values(ascending=False).iplot(title="Number of claim initiations by Month",xTitle="Month",yTitle="Number of claim initiations")


## Insurance company market value points have a mean of 3.7k while having a standard deviation of 6k which is explained by the 25th and 50th percentile values (0). So the values are skewed towards the higher end. 75 percent of all values are under almost 9k while the max is at ~30k. 
## Most devices are situated below 10k OR have depreciated in value as a result of age.

In [1021]:
ic_df.ic_market_value.describe()

count      724.000000
mean      3765.023481
std       6000.611529
min          0.000000
25%          0.000000
50%          0.000000
75%       8990.000000
max      29990.000000
Name: ic_market_value, dtype: float64

In [921]:
ic_df.ic_market_value.iplot("hist",bins=10,title="Handset Market Values",xTitle="Market Value",yTitle="Counts")

## For modelling purposes, we would further need to log-transform some variables while standardizing and normalizing a lot of them. We might have to use some synthetic data points in cases of class imbalance. And impute NULLs by computation or naive logic if the columns are important.


                                            ## The end ##